<a href="https://colab.research.google.com/github/raghuch/EVA8_assignments/blob/main/assignment_2p5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
from __future__ import print_function
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [36]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)



    def forward(self, x, int_input):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        pred = F.log_softmax(x)
        return pred, (pred.argmax(dim=1, keepdim=True) + int_input)

In [43]:
# !pip install torchsummary
# from torchsummary import summary
# use_cuda = torch.cuda.is_available()
# device = torch.device("cuda" if use_cuda else "cpu")
# model = Net().to(device)
# summary(model, input_size=((1, 28, 28), (1)))

In [44]:

torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [45]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        int_input = torch.randint(10, (data.shape[0], 1)).to(device)
        optimizer.zero_grad()
        output, sum = model(data, int_input)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            int_input = torch.randint(10, (data.shape[0], 1)).to(device)
            output, sum = model(data, int_input)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [46]:
model = Net().to(device)

In [47]:
device

device(type='cuda')

In [48]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(10):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-36-c8f9f68f97da>:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = F.log_softmax(x)
loss=0.07660208642482758 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.53it/s]



Test set: Average loss: 0.0663, Accuracy: 9782/10000 (98%)



loss=0.046160366386175156 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.38it/s]



Test set: Average loss: 0.0389, Accuracy: 9873/10000 (99%)



loss=0.011607259511947632 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.28it/s]


...


Test set: Average loss: 0.0257, Accuracy: 9928/10000 (99%)



loss=0.021846042945981026 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.93it/s]



Test set: Average loss: 0.0371, Accuracy: 9892/10000 (99%)



loss=0.001793260220438242 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.38it/s]



Test set: Average loss: 0.0301, Accuracy: 9913/10000 (99%)



In [34]:


for i, c in test_loader:
    out = model(i.to(device))
    pred = out.argmax(dim=1, keepdim=True)
    int_input = torch.randint(10, (batch_size, 1)).to(device)
    print(f'pred: {pred.T}, rand_int: {int_input.T}, sum: {pred.T+int_input.T}')
    #print(+torch.randint(10, (batch_size, 1)))

<ipython-input-3-456f7f5965cb>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


pred: tensor([[6, 0, 3, 9, 6, 1, 3, 7, 0, 6, 1, 4, 9, 4, 9, 8, 4, 5, 7, 5, 8, 7, 3, 4,
         3, 6, 4, 3, 1, 1, 9, 9, 3, 0, 4, 7, 6, 2, 8, 8, 5, 2, 3, 3, 5, 0, 1, 8,
         3, 4, 7, 0, 2, 1, 7, 8, 1, 7, 9, 3, 8, 6, 7, 7, 8, 6, 7, 1, 2, 8, 6, 0,
         3, 6, 0, 7, 7, 9, 4, 7, 5, 7, 3, 2, 8, 9, 6, 8, 9, 3, 8, 5, 5, 1, 3, 1,
         9, 6, 4, 7, 3, 8, 1, 0, 9, 1, 6, 1, 9, 7, 7, 6, 2, 4, 7, 7, 8, 3, 2, 0,
         0, 7, 2, 3, 0, 4, 3, 8]], device='cuda:0'), rand_int: tensor([[5, 5, 7, 4, 7, 3, 9, 3, 1, 0, 2, 0, 0, 5, 4, 1, 4, 8, 9, 3, 7, 5, 8, 5,
         7, 7, 7, 8, 0, 3, 5, 5, 3, 7, 8, 5, 3, 6, 5, 1, 2, 0, 7, 6, 5, 4, 8, 9,
         2, 3, 7, 6, 6, 0, 7, 3, 8, 1, 8, 8, 9, 7, 1, 0, 4, 0, 4, 5, 8, 8, 2, 1,
         7, 3, 1, 3, 8, 0, 5, 6, 0, 5, 8, 7, 4, 1, 3, 1, 1, 9, 5, 0, 5, 4, 4, 1,
         5, 7, 9, 3, 4, 1, 8, 9, 5, 4, 4, 1, 6, 5, 6, 6, 6, 8, 0, 8, 3, 9, 5, 8,
         9, 2, 9, 1, 1, 5, 1, 0]], device='cuda:0'), sum: tensor([[11,  5, 10, 13, 13,  4, 12, 10,  1,  6,  3,  4,  9,  9

RuntimeError: ignored